In [ ]:
# TODO: 
# Do analysis for 2008 data

import pandas as pd
import numpy as np
import statsmodels.api as sm

import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [ ]:
gini = pd.read_csv('./data/gini.csv')

print(gini.shape)

gini18 = gini[["Country Name", "Country Code", "2017", "2018", "2019"]]
print(gini18.shape)

# filter out rows with missing all 2017, 2018 and 2019 values
gini18 = gini18.dropna(subset=["2017", "2018", "2019"], how='all')
print(gini18.shape)

# if 2018 is missing, use 2017 and 2019 to fill it
# gini18["2018"] = gini18["2018"].fillna((gini18["2017"] + gini18["2019"]) / 2)
# if gini18["2018"].isna():
#     gini18["2018"] = gini18["2017"] if gini18["2018"].isna() else gini18["2018"]
    
# if 2018 is missing, use 2017 value to fill it, if 2017 is also missing, use 2019 value
gini18["2018"] = gini18["2018"].fillna(gini18["2017"])
gini18["2018"] = gini18["2018"].fillna(gini18["2019"])


print(gini18.shape)

In [ ]:

incomet = pd.read_csv('./data/income_top_10.csv')
incomet = incomet[["Country Name", "Country Code", "2017", "2018", "2019"]]

print(incomet.shape)
incomet = incomet.dropna(subset=["2017", "2018", "2019"], how='all')

# incomet["2018"] = incomet["2018"].fillna((incomet["2017"] + incomet["2019"]) / 2)
incomet["2018"] = incomet["2018"].fillna(incomet["2017"])
incomet["2018"] = incomet["2018"].fillna(incomet["2019"])

print(incomet.shape)

In [ ]:
incomeb = pd.read_csv('./data/income_bot_10.csv')
incomeb = incomeb[["Country Name", "Country Code", "2017", "2018", "2019"]]
print(incomeb.shape)
incomeb = incomeb.dropna(subset=["2017", "2018", "2019"], how='all')
incomeb["2018"] = incomeb["2018"].fillna(incomeb["2017"])
incomeb["2018"] = incomeb["2018"].fillna(incomeb["2019"])
print(incomeb.shape)

In [ ]:
# Reading the regressor dataframes
gdp = pd.read_csv('./data/gdp_per_capita.csv')
lit = pd.read_csv('./data/literacy_rate.csv')
lif = pd.read_csv('./data/life_expectancy.csv')
urb = pd.read_csv('./data/urban_population_percentage.csv')
pop = pd.read_csv('./data/population_density.csv')

In [ ]:
def clean_up(target, frames):
    """
    target: the dataframe to be used for cleaning
    frames: the list of dataframes to be cleaned
    
    Returns X, y where X is the feature matrix combining all the dataframes in frames and y is the target dataframe
    """
    countries = target["Country Code"]
    print(countries.shape)
    X = pd.DataFrame()
    for name, frame in frames.items():
        # for each frame, filter out the rows that are not in the countries list
        # print(frame.columns)
        frame = frame[["Country Code", "2017", "2018", "2019"]]
        frame = frame[frame["Country Code"].isin(countries)]
        # now, check if atleast one of 2017, 2018, 2019 is missing
        frame = frame.dropna(subset=["2017", "2018", "2019"], how='all')
        # if 2018 is missing, use 2017 and 2019 to fill it
        frame[name] = frame["2018"].fillna((frame["2017"] + frame["2019"]) / 2)
        
        # drop 2018, 2019, 2017 columns from frame
        frame = frame.drop(columns=["2017", "2018", "2019"])
        # print(frame.shape)
        # concat the frame with X
        # if len(X) == 0:
        #     X = frame
        # else:
        #     X = pd.merge(X, frame, on="Country Code", how="outer")
        X = pd.concat([X, frame], axis=1)
    
    # filter out all rows from X that has any missing value
    
    X = pd.concat([X, target["2018"]], axis=1)
    
    X = X.dropna()
    # only keep one Country Code col and drop the other two in X
    X = X.drop(columns=["Country Code"])
    
    
    # extract last column from X and drop that col from X as well
    y = X["2018"]
    X = X.drop(columns=["2018"])
    # y = target["2018"]
    return X, y


## Gini index

In [ ]:
frames = {
    "gdp": gdp,
    # "lit": lit,
    "lif": lif,
    "urb": urb,
    "pop": pop
}
X, y = clean_up(gini18, frames)
print(X.shape, y.shape)

est2 = sm.add_constant(X)
est = sm.OLS(y, est2).fit()
print(est.summary())

In [ ]:
# Transform gdp to log gdp
# X["gdp"] = np.log(X["gdp"]) 

X["log_gdp"] = np.log(X["gdp"])
X["gdp_2"] = X["gdp"] ** 2

# gdp + gdp_2 = 0.218
# gdp = 0.18
# gdp + log gdp = 0.195
# gdp + log gdp + gdp_2 = 0.219
# gdp + gdp_2 + gdp_3 = 0.071

print(X.columns)
est2 = sm.add_constant(X)
est = sm.OLS(y, est2).fit()
print(est.summary())
    

In [ ]:
# Heteroskedasticity test

# Plot the residual error terms against the y_pred
y_pred = est.predict(est2)

residuals = y - y_pred
plt.scatter(y_pred, residuals)
plt.xlabel("Predicted values")
plt.ylabel("Residuals")
plt.title("Residuals vs Fitted")
plt.show()

In [ ]:
# Variance Inflation Factor

# Calculate the VIF for each estimated coefficient using statsmodels package
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif

In [ ]:
frames = {
    "gdp": gdp,
    # "lit": lit,
    "lif": lif,
    # "urb": urb,
    "pop": pop
}
X, y = clean_up(gini18, frames)
print(X.shape, y.shape)

est2 = sm.add_constant(X)
est = sm.OLS(y, est2).fit()
print(est.summary())

Addition of new variables makes the model explain better (increments of R2 consistent with increase in number of variables).

## Income top 10% 

In [ ]:
frames = {
    "gdp": gdp,
    # "lit": lit,
    "lif": lif,
    "urb": urb, 
    "pop": pop
}
X, y = clean_up(incomet, frames)
print(X.shape, y.shape)

est2 = sm.add_constant(X)
est = sm.OLS(y, est2).fit()
print(est.summary())

In [ ]:
# Transform gdp to log gdp
# X["gdp"] = np.log(X["gdp"]) 

X["log_gdp"] = np.log(X["gdp"])
X["gdp_2"] = X["gdp"] ** 2

# gdp = 0.19
# gdp + log_gdp = 0.226
# gdp + gdp_2 = 0.230
# gdp + log_gdp + gdp_2 = 0.241

print(X.columns)
est2 = sm.add_constant(X)
est = sm.OLS(y, est2).fit()
print(est.summary())
    

In [ ]:
# Heteroskedasticity test

# Plot the residual error terms against the y_pred
y_pred = est.predict(est2)

residuals = y - y_pred
plt.scatter(y_pred, residuals)
plt.xlabel("Predicted values")
plt.ylabel("Residuals")
plt.title("Residuals vs Fitted")
plt.show()

In [ ]:
# Variance Inflation Factor

# Calculate the VIF for each estimated coefficient using statsmodels package
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif

In [ ]:
frames = {
    "gdp": gdp,
    # "lit": lit,
    # "lif": lif,
    # "urb": urb, 
    # "pop": pop
}
X, y = clean_up(incomet, frames)
print(X.shape, y.shape)

est2 = sm.add_constant(X)
est = sm.OLS(y, est2).fit()
print(est.summary())

# Income bottom 10%

In [ ]:
frames = {
    "gdp": gdp,
    # "lit": lit,
    "lif": lif,
    "urb": urb,
    "pop": pop
}
X, y = clean_up(incomeb, frames)
print(X.shape, y.shape)

est2 = sm.add_constant(X)
est = sm.OLS(y, est2).fit()
print(est.summary())

In [ ]:
# Transform gdp to log gdp
# X["gdp"] = np.log(X["gdp"]) 

X["log_gdp"] = np.log(X["gdp"])
X["gdp_2"] = X["gdp"] ** 2
# X["gdp_3"] = X["gdp"] ** 3

# gdp = 0.137
# gdp + log gdp = 0.139
# gdp + gdp_2 = 0.147
# gdp + log gdp + gdp_2 = 0.160
# gdp + gdp_3 = 0.037


print(X.columns)
est2 = sm.add_constant(X)
est = sm.OLS(y, est2).fit()
print(est.summary())
    

In [ ]:
# Heteroscedasticity Test

# Plot the residual error terms against the y_pred
y_pred = est.predict(est2)

residuals = y - y_pred
plt.scatter(y_pred, residuals)
plt.xlabel("Predicted values")
plt.ylabel("Residuals")
plt.title("Residuals vs Fitted")
plt.show()

In [ ]:
# Variance Inflation Factor

# Calculate the VIF for each estimated coefficient using statsmodels package
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif

In [ ]:
frames = {
    "gdp": gdp,
    # "lit": lit,
    # "lif": lif,
    # "urb": urb,
    "pop": pop
}
X, y = clean_up(incomeb, frames)
print(X.shape, y.shape)

est2 = sm.add_constant(X)
est = sm.OLS(y, est2).fit()
print(est.summary())

Drastic decrease in explanation if urb is removed.

# Discussion